In [ ]:
#tokenize words and phrases for use in Terms Relevance and related
# import libraries

#Import Libraries
import pandas as pd
import numpy as np
from pathlib import Path

# Initial imports
import nltk
from nltk.corpus import reuters
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
#load data
# Load the review data
reviews_df=pd.read_csv('glassdoor_reviews.csv')

# Preview column names
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422180 entries, 0 to 422179
Data columns (total 28 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   id                                             422180 non-null  int64  
 1   index                                          343221 non-null  float64
 2   reviews.val.cons                               343214 non-null  object 
 3   reviews.val.date                               343221 non-null  object 
 4   reviews.val.featured                           343221 non-null  object 
 5   reviews.val.helpfulCount                       343221 non-null  float64
 6   reviews.val.id                                 343221 non-null  float64
 7   reviews.val.pros                               343221 non-null  object 
 8   reviews.val.publishedOn                        343221 non-null  object 
 9   reviews.val.publisher                

In [26]:
# Load the fact data
full_df=pd.read_csv('glassdoor.csv')
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165290 entries, 0 to 165289
Columns: 163 entries, benefits.benefitRatingDecimal to wwfu
dtypes: bool(11), float64(35), int64(23), object(94)
memory usage: 193.4+ MB


In [20]:
# Preview column names
full_df.columns

Index(['benefits.benefitRatingDecimal', 'benefits.comments',
       'benefits.highlights', 'benefits.numRatings',
       'benefits.employerSummary', 'breadCrumbs', 'gaTrackerData.category',
       'gaTrackerData.empId', 'gaTrackerData.empName', 'gaTrackerData.empSize',
       ...
       'salary.currency.displayName', 'salary.currency.id',
       'salary.currency.name', 'salary.currency.negativeTemplate',
       'salary.currency.new', 'salary.currency.positiveTemplate',
       'salary.currency.symbol', 'salary.lastSalaryDate', 'salary.salaries',
       'wwfu'],
      dtype='object', length=163)

In [27]:
#todo - join with ountry /language data to eliminate non-english roles and reviews
glassdoor_df = pd.merge(
    full_df,
    reviews_df,
    how= 'inner',
    left_on= 'gaTrackerData.empId',
    right_on='id',
    left_index = True,
    right_index  = False,
    sort = False,
    suffixes=('_x', '_y'),
    copy = True,
    indicator  = False,
    validate=None,
)
glassdoor_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155044 entries, 243221 to 4359
Columns: 191 entries, benefits.benefitRatingDecimal to reviews.val.reviewResponses
dtypes: bool(11), float64(48), int64(24), object(108)
memory usage: 215.7+ MB


In [30]:
glassdoor_df.head(10)

,benefits.benefitRatingDecimal,benefits.comments,benefits.highlights,benefits.numRatings,benefits.employerSummary,breadCrumbs,gaTrackerData.category,gaTrackerData.empId,gaTrackerData.empName,gaTrackerData.empSize,...,reviews.val.reviewerJobTitle,reviews.val.reviewerLocation,reviews.val.reviewerStatus,reviews.val.summaryPoints.ceoApproval,reviews.val.summaryPoints.outlook,reviews.val.summaryPoints.recommend,reviews.val.title,reviews.val.adviceToManagement,reviews.val.companyResponse,reviews.val.reviewResponses
243221,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,Systems Engineer,Belgrade (Serbia),Current Employee,1.0,2.0,1.0,Enterprise with a soul,"Keep this trends, and just consider more man p...",NaN,NaN
243222,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,Lead Technical Analyst,Belgrade (Serbia),Current Employee,1.0,1.0,1.0,Nicee,NaN,Thank you for your review. Glad to see you lov...,1696.0
243223,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,Consulting Team Generalist,Belgrade (Serbia),Current Employee,0.0,2.0,0.0,Employed in SAP,NaN,NaN,NaN
243224,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,SAP Consultant,Belgrade (Serbia),Current Employee,0.0,0.0,0.0,Great company but not focused on employees,You should focus more on the people. There are...,NaN,NaN
243225,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,Business Support Specialist,Belgrade (Serbia),Current Employee,1.0,0.0,1.0,Challangable,NaN,NaN,NaN
243221,4.0,54846.0,54846.0,3,NaN,54846,-1,10416,Boehringer Ingelheim,10000--1,...,Systems Engineer,Belgrade (Serbia),Current Employee,1.0,2.0,1.0,Enterprise with a soul,"Keep this trends, and just consider more man p...",NaN,NaN
243222,4.0,54846.0,54846.0,3,NaN,54846,-1,10416,Boehringer Ingelheim,10000--1,...,Lead Technical Analyst,Belgrade (Serbia),Current Employee,1.0,1.0,1.0,Nicee,NaN,Thank you for your review. Glad to see you lov...,1696.0
243223,4.0,54846.0,54846.0,3,NaN,54846,-1,10416,Boehringer Ingelheim,10000--1,...,Consulting Team Generalist,Belgrade (Serbia),Current Employee,0.0,2.0,0.0,Employed in SAP,NaN,NaN,NaN
243224,4.0,54846.0,54846.0,3,NaN,54846,-1,10416,Boehringer Ingelheim,10000--1,...,SAP Consultant,Belgrade (Serbia),Current Employee,0.0,0.0,0.0,Great company but not focused on employees,You should focus more on the people. There are...,NaN,NaN
243225,4.0,54846.0,54846.0,3,NaN,54846,-1,10416,Boehringer Ingelheim,10000--1,...,Business Support Specialist,Belgrade (Serbia),Current Employee,1.0,0.0,1.0,Challangable,NaN,NaN,NaN


In [29]:
glassdoor_df.columns

Index(['benefits.benefitRatingDecimal', 'benefits.comments',
       'benefits.highlights', 'benefits.numRatings',
       'benefits.employerSummary', 'breadCrumbs', 'gaTrackerData.category',
       'gaTrackerData.empId', 'gaTrackerData.empName', 'gaTrackerData.empSize',
       ...
       'reviews.val.reviewerJobTitle', 'reviews.val.reviewerLocation',
       'reviews.val.reviewerStatus', 'reviews.val.summaryPoints.ceoApproval',
       'reviews.val.summaryPoints.outlook',
       'reviews.val.summaryPoints.recommend', 'reviews.val.title',
       'reviews.val.adviceToManagement', 'reviews.val.companyResponse',
       'reviews.val.reviewResponses'],
      dtype='object', length=191)

In [31]:
#drop NANs in reviews.val.date - (only keeping rows with roiews)
glassdoor_df.dropna(
    axis=0,
    how='any',
    subset=['reviews.val.pros'],
    inplace=True,
)

In [32]:
glassdoor_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128908 entries, 243221 to 4359
Columns: 191 entries, benefits.benefitRatingDecimal to reviews.val.reviewResponses
dtypes: bool(11), float64(48), int64(24), object(108)
memory usage: 179.4+ MB


In [34]:
#replace '.' with '_' in column headers
glassdoor_df.columns = glassdoor_df.columns.str.replace("[.]", "_")
glassdoor_df.head()

,benefits_benefitRatingDecimal,benefits_comments,benefits_highlights,benefits_numRatings,benefits_employerSummary,breadCrumbs,gaTrackerData_category,gaTrackerData_empId,gaTrackerData_empName,gaTrackerData_empSize,...,reviews_val_reviewerJobTitle,reviews_val_reviewerLocation,reviews_val_reviewerStatus,reviews_val_summaryPoints_ceoApproval,reviews_val_summaryPoints_outlook,reviews_val_summaryPoints_recommend,reviews_val_title,reviews_val_adviceToManagement,reviews_val_companyResponse,reviews_val_reviewResponses
243221,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,Systems Engineer,Belgrade (Serbia),Current Employee,1.0,2.0,1.0,Enterprise with a soul,"Keep this trends, and just consider more man p...",NaN,NaN
243222,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,Lead Technical Analyst,Belgrade (Serbia),Current Employee,1.0,1.0,1.0,Nicee,NaN,Thank you for your review. Glad to see you lov...,1696.0
243223,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,Consulting Team Generalist,Belgrade (Serbia),Current Employee,0.0,2.0,0.0,Employed in SAP,NaN,NaN,NaN
243224,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,SAP Consultant,Belgrade (Serbia),Current Employee,0.0,0.0,0.0,Great company but not focused on employees,You should focus more on the people. There are...,NaN,NaN
243225,4.0,2802.0,2802.0,3,NaN,2802,-1,10416,Boehringer Ingelheim,10000--1,...,Business Support Specialist,Belgrade (Serbia),Current Employee,1.0,0.0,1.0,Challangable,NaN,NaN,NaN


In [35]:
glassdoor_df.columns

Index(['benefits_benefitRatingDecimal', 'benefits_comments',
       'benefits_highlights', 'benefits_numRatings',
       'benefits_employerSummary', 'breadCrumbs', 'gaTrackerData_category',
       'gaTrackerData_empId', 'gaTrackerData_empName', 'gaTrackerData_empSize',
       ...
       'reviews_val_reviewerJobTitle', 'reviews_val_reviewerLocation',
       'reviews_val_reviewerStatus', 'reviews_val_summaryPoints_ceoApproval',
       'reviews_val_summaryPoints_outlook',
       'reviews_val_summaryPoints_recommend', 'reviews_val_title',
       'reviews_val_adviceToManagement', 'reviews_val_companyResponse',
       'reviews_val_reviewResponses'],
      dtype='object', length=191)

In [36]:
# Count the total number of reviews in the reviews.val.pros column
print(f"Total number of docs in the corpus: {len(glassdoor_df.reviews_val_id)}")

Total number of docs in the corpus: 128908


In [84]:
# Get bag of words from single doc -  Select and print a row of text
glassdoor_df.reviews_val_pros

243221    We all have to agree that working for large en...
243222        Company culture, possition politics, projects
243223    Positive, forward facing new technologies envi...
243224    The largest business software provider in the ...
243225                 Flexibility, Virtual work, team work
                                ...                        
4355         A lot of hard-working and ambitious colleagues
4356               Strong training session,  Great teamwork
4357      Perhaps it is that Amazon is a great springboard.
4358      - Culture is vv excellent, vv open \r\n- Work ...
4359      All colleagues very very friendly, can just go...
Name: reviews_val_pros, Length: 128908, dtype: object

In [100]:
# Create the CountVectorizer instance defining the stopwords in English to be ignored
vectorizer = CountVectorizer(stop_words="english")
vectorizer

CountVectorizer(stop_words='english')

In [103]:
# Get the tokenization and occurrence count
X = vectorizer.fit_transform(glassdoor_df.reviews_val_pros)

In [104]:
# Retrieve unique words list
words = vectorizer.get_feature_names()
print(words)

['00', '000', '01', '10', '100', '1000', '101', '11', '111', '12', '120', '120k', '12pm', '13', '13em', '13eme', '13ème', '14', '140', '15', '1500', '15h45', '15lacs', '16', '164', '17', '17h30', '17h45', '18', '180', '1800', '19', '1900', '1950', '1s', '1st', '1x', '1year', '20', '200', '2006', '2007', '2008', '2009', '200h', '2010', '2013', '2016', '2017', '2018', '2019', '2100', '23', '24', '247', '25', '25th', '27', '28', '2h', '2nd', '2ndwatch', '2weeks', '30', '30am', '310', '32', '35', '35h', '360', '37', '38', '3d', '3rd', '40', '401', '401k', '430pm', '44', '45', '45min', '48', '4free', '4h', '4pm', '4th', '4yrs', '50', '500', '51', '52nd', '5g', '5mois', '5th', '60', '600pm', '6p', '6pm', '7000', '730am', '75', '75th', '78', '7h00', '7k', '80', '85', '8pm', '90', '900am', '996', '9gag', '9h', '9h00', 'aaa', 'aan', 'aandacht', 'aangename', 'aankaarten', 'aansloot', 'aantal', 'abarcar', 'abbandona', 'abbau', 'abele', 'aberta', 'aberto', 'abertos', 'abierta', 'abierto', 'abierto

In [105]:
# X raw data contains the occurrence of each term in the document. A unique ID is assigned to each term.
print(X)

  (0, 383)	1
  (0, 10367)	3
  (0, 5410)	2
  (0, 3193)	1
  (0, 6943)	2
  (0, 8985)	1
  (0, 1829)	2
  (0, 5645)	2
  (0, 6301)	3
  (0, 9388)	1
  (0, 7131)	1
  (0, 1730)	1
  (0, 2829)	1
  (0, 796)	1
  (0, 5744)	3
  (0, 10357)	1
  (0, 4521)	1
  (0, 1495)	1
  (0, 4338)	1
  (0, 8269)	1
  (0, 1079)	1
  (0, 8560)	1
  (0, 6669)	1
  (0, 1513)	1
  (0, 2014)	1
  :	:
  (128904, 9552)	1
  (128904, 9004)	1
  (128904, 9270)	1
  (128904, 8474)	1
  (128905, 4271)	1
  (128905, 497)	1
  (128905, 8832)	1
  (128906, 10357)	1
  (128906, 2332)	1
  (128906, 6589)	1
  (128906, 3954)	1
  (128906, 10362)	1
  (128906, 5846)	1
  (128906, 3426)	1
  (128906, 3973)	1
  (128906, 10154)	2
  (128907, 721)	1
  (128907, 5241)	1
  (128907, 7666)	1
  (128907, 6589)	1
  (128907, 3954)	1
  (128907, 1793)	1
  (128907, 9194)	1
  (128907, 5746)	1
  (128907, 5878)	1


In [106]:
# Get the bag of words as DataFrame with word counts
words_df = pd.DataFrame(
    list(zip(words, np.ravel(X.sum(axis=0)))), columns=["Word", "Word_Count"]
)
words_df

,Word,Word_Count
0,00,2494
1,000,1
2,01,2494
3,10,305
4,100,215
...,...,...
10530,überhaupt,2
10531,übernimmt,18
10532,überstunden,10
10533,überwiegend,10


In [107]:
#Calculating the TF-IDF from all 'Pro' comments
# Get the corpus (first 1000 files from dataset - will do more when on Kaggle/Google)
all_docs_id = glassdoor_df.reviews_val_id
all_docs_id

243221    29948693.0
243222    25433141.0
243223     1826212.0
243224     1777696.0
243225      796163.0
             ...    
4355      25039731.0
4356      28884763.0
4357      26241942.0
4358      25021906.0
4359      22624861.0
Name: reviews_val_id, Length: 128908, dtype: object

In [108]:
corpus_id = all_docs_id[0:1000].astype(int)
corpus_id

ValueError: invalid literal for int() with base 10: '29948693.0'

In [125]:
corpus = glassdoor_df.reviews_val_pros
print(corpus)

243221    We all have to agree that working for large en...
243222        Company culture, possition politics, projects
243223    Positive, forward facing new technologies envi...
243224    The largest business software provider in the ...
243225                 Flexibility, Virtual work, team work
                                ...                        
4355         A lot of hard-working and ambitious colleagues
4356               Strong training session,  Great teamwork
4357      Perhaps it is that Amazon is a great springboard.
4358      - Culture is vv excellent, vv open \r\n- Work ...
4359      All colleagues very very friendly, can just go...
Name: reviews_val_pros, Length: 128908, dtype: object


In [126]:
#Getting the TF-IDF
vectorizer = TfidfVectorizer(stop_words="english")
X_corpus = vectorizer.fit_transform(corpus)

In [127]:
# Getting matrix info
print(f"Matrix shape: {X_corpus.shape}")
print(f"Total number of documents: {X_corpus.shape[0]}")
print(f"Total number of unique words (tokens): {X_corpus.shape[1]}")

Matrix shape: (128908, 10535)
Total number of documents: 128908
Total number of unique words (tokens): 10535


In [128]:
# Retrieve words list from corpus
words_corpus = vectorizer.get_feature_names()
print(words_corpus)

['00', '000', '01', '10', '100', '1000', '101', '11', '111', '12', '120', '120k', '12pm', '13', '13em', '13eme', '13ème', '14', '140', '15', '1500', '15h45', '15lacs', '16', '164', '17', '17h30', '17h45', '18', '180', '1800', '19', '1900', '1950', '1s', '1st', '1x', '1year', '20', '200', '2006', '2007', '2008', '2009', '200h', '2010', '2013', '2016', '2017', '2018', '2019', '2100', '23', '24', '247', '25', '25th', '27', '28', '2h', '2nd', '2ndwatch', '2weeks', '30', '30am', '310', '32', '35', '35h', '360', '37', '38', '3d', '3rd', '40', '401', '401k', '430pm', '44', '45', '45min', '48', '4free', '4h', '4pm', '4th', '4yrs', '50', '500', '51', '52nd', '5g', '5mois', '5th', '60', '600pm', '6p', '6pm', '7000', '730am', '75', '75th', '78', '7h00', '7k', '80', '85', '8pm', '90', '900am', '996', '9gag', '9h', '9h00', 'aaa', 'aan', 'aandacht', 'aangename', 'aankaarten', 'aansloot', 'aantal', 'abarcar', 'abbandona', 'abbau', 'abele', 'aberta', 'aberto', 'abertos', 'abierta', 'abierto', 'abierto

In [129]:
# Getting the TF-IDF weight of each word in corpus as DataFrame
words_corpus_df = pd.DataFrame(
    list(zip(words_corpus, np.ravel(X_corpus.mean(axis=0)))), columns=["Word", "TF-IDF"]
)

words_corpus_df = words_corpus_df.sort_values(by=["TF-IDF"], ascending=False)

In [130]:
# Highest 10 TF-IDF scores
words_corpus_df.head(10)


,Word,TF-IDF
4213,good,0.059985
10357,work,0.048583
4271,great,0.039417
6918,people,0.036076
3251,environment,0.030120
1898,company,0.029565
2332,culture,0.027559
10367,working,0.027514
6348,nice,0.025635
1079,benefits,0.021717


In [131]:
# Lowest 10 TF-IDF scores
words_corpus_df.tail(10)

,Word,TF-IDF
8801,spielraum,5.931616e-07
10298,wichtigste,5.931616e-07
2522,denen,5.931616e-07
1089,bereut,5.931616e-07
4321,grund,5.931616e-07
1016,bedingt,5.931616e-07
10339,wo,5.931616e-07
4369,habe,5.931616e-07
8558,sicherlich,5.931616e-07
8400,seit,5.931616e-07


In [133]:
#to do - create stop list of words that are relativel meaningless (i.e. work, working, great good)
drop_numbers = ['00', '000', '01', '10', '100', '1000', '101', '11', '111', '12', '120', '120k', '12pm', '13', '13em', '13eme', '13ème', '14', '140', '15', '1500', '15h45', '15lacs', '16', '164', '17', '17h30', '17h45', '18', '180', '1800', '19', '1900', '1950', '1s', '1st', '1x', '1year', '20', '200', '2006', '2007', '2008', '2009', '200h', '2010', '2013', '2016', '2017', '2018', '2019', '2100', '23', '24', '247', '25', '25th', '27', '28', '2h', '2nd', '2ndwatch', '2weeks', '30', '30am', '310', '32', '35', '35h', '360', '37', '38', '3d', '3rd', '40', '401', '401k', '430pm', '44', '45', '45min', '48', '4free', '4h', '4pm', '4th', '4yrs', '50', '500', '51', '52nd', '5g', '5mois', '5th', '60', '600pm', '6p', '6pm', '7000', '730am', '75', '75th', '78', '7h00', '7k', '80', '85', '8pm', '90', '900am', '996', '9gag', '9h', '9h00']
